In [1]:
import psycopg2
import pandas as pd
import os

### Pasos para generar variable de entorno en MAC/LINUX

- Abrir tu terminal
- Tipear el siguiente comando en el terminal y ejecutarlo:  
  ```bash
  nano ~/.zshrc
- Colocar la variable de entorno al final del archivo
  ```bash
  export MI_VARIABLE="mi_valor"
- Luego de colocar la variable, presiona CTRL + O, luego Enter, y CTRL + X para salir.
- Tipear y ejecutar el siguiente comando en el terminal
  ```bash
  source ~/.zshrc
- Verificar que la variable se haya definido
  ```bash
  echo $MI_VARIABLE
- Abrir python y usar el paquete os para importar tu variable de entorno





### 🌐 Usando Python para subir datos

Para completar la misión, el equipo desea automatizar la carga de datos usando Python. Aquí usamos la librería psycopg2 para subir información desde archivos .csv.

In [5]:
# Establecer la conexión a la base de datos
def conectar_a_base_de_datos():
    conn_info = {
        "host": "localhost",  # Cambia esto si tu base no está en localhost
        "database": "techmartdb",  #OJO : siempre en minúsculas
        "user": "postgres",
        "password": os.getenv('ypass_psql'),
        "port": 5432  # Puerto por defecto
    }
    
    try:
        # Establecer conexión con la base de datos
        conn = psycopg2.connect(**conn_info)
        print("Conexión exitosa a la base de datos.")
        return conn
    except Exception as e:
        print(f"Error al conectar a la base de datos: {e}")
        return None

# Conectar a la base de datos
conn = conectar_a_base_de_datos()

Conexión exitosa a la base de datos.


In [7]:
# Leer datos desde un archivo CSV
df = pd.read_csv("./data/productos_nuevos.csv")

In [8]:
df

,ProductName,UnitPrice,UnitsInStock,UnitsOnOrder,DateAdded
0,Laptop Pro,1200.50,15,5,2024-11-25
1,Teclado Mecánico,85.75,50,10,2024-12-28
2,Monitor Ultra HD,320.00,8,20,2024-11-20
3,Mouse Gamer,45.90,100,30,2024-12-01
4,Audífonos Inalámbricos,65.30,60,15,2024-10-17
5,Impresora Multifunción,150.00,20,5,2025-01-06
6,Tablet 10 pulgadas,250.75,30,12,2024-12-23
7,Cámara Web 1080p,70.00,40,8,2024-10-09
8,Disco Duro Externo 1TB,110.50,25,10,2024-09-27
9,Router WiFi 6,90.00,15,4,2024-08-12


In [10]:
# Insertar datos en la tabla Productos
cursor = conn.cursor()
for index, row in df.iterrows():
    cursor.execute("""
        INSERT INTO Productos (ProductName, UnitPrice, UnitsInStock, UnitsOnOrder, DateAdded)
        VALUES (%s, %s, %s, %s, %s)
    """, (row['ProductName'], row['UnitPrice'], row['UnitsInStock'], row['UnitsOnOrder'], row['DateAdded']))

conn.commit()
cursor.close()
conn.close()

### 🔄 Opciones para reemplazar datos en PostgreSQL con Python


🔹 Opción 1: Eliminar todos los datos antes de insertar (TRUNCATE)


In [16]:
# Conexión a PostgreSQL
conn = psycopg2.connect(
    dbname="techmartdb",
    user="postgres",
    password= os.getenv('ypass_psql'),
    host="localhost",
    port="5432"
)
cur = conn.cursor()


In [17]:
# Borrar todos los registros de la tabla antes de insertar
cur.execute("TRUNCATE TABLE productos RESTART IDENTITY;")
conn.commit()

**📝 Explicación:**

- TRUNCATE TABLE productos RESTART IDENTITY; borra todos los registros y reinicia los IDs para que empiecen desde 1.
- Luego, el script inserta los nuevos datos.


In [13]:
# Insertar los datos en PostgreSQL
for _, row in df.iterrows():
    cur.execute(
        "INSERT INTO productos (productname, unitprice, unitsinstock, unitsonorder, dateadded) VALUES (%s, %s, %s, %s, %s);",
        (row["ProductName"], row["UnitPrice"], row["UnitsInStock"], row["UnitsOnOrder"], row["DateAdded"])
    )

In [14]:
conn.commit()
cur.close()
conn.close()

🔹 Opción 2: Actualizar datos si existen, insertar si no (UPSERT con ON CONFLICT)

 *🛠️ Asegurar que `ProductName` sea único en la tabla*

Para evitar duplicados en la columna `ProductName`, agregamos una restricción `UNIQUE` en la base de datos PostgreSQL.

```sql
ALTER TABLE productos ADD CONSTRAINT unique_productname UNIQUE (productname);


In [19]:
# Insertar o actualizar datos
for _, row in df.iterrows():
    cur.execute("""
        INSERT INTO productos (productname, unitprice, unitsinstock, unitsonorder, dateadded)
        VALUES (%s, %s, %s, %s, %s)
        ON CONFLICT (productname) DO UPDATE SET 
            unitprice = EXCLUDED.unitprice,
            unitsinstock = EXCLUDED.unitsinstock,
            unitsonorder = EXCLUDED.unitsonorder,
            dateadded = EXCLUDED.dateadded;
    """, (row["ProductName"], row["UnitPrice"], row["UnitsInStock"], row["UnitsOnOrder"], row["DateAdded"]))



In [20]:
conn.commit()
cur.close()
conn.close()

**✅ Explicación:**

🔹 **ON CONFLICT (productname)**: Si `ProductName` ya existe, en vez de insertar, **actualiza los valores**.  
🔹 **Si no existe**, lo inserta como un **nuevo producto**.  
